## Import Modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
import random
import tensorflow as tf
import PIL
import seaborn as sns
from tensorflow import keras


# tf.debugging.set_log_device_placement(True)
warnings.filterwarnings('ignore')
print(tf.__version__)
print(keras.__version__)


## Create Dataframe for Input and Output

In [ ]:
input_path = []
label = []
main_class_names = ['Cat', 'Dog', 'Horse']
image_path = 'images_download'

for class_name in os.listdir(image_path):
  for path in os.listdir(f'{image_path}/{class_name}'):
    if class_name == 'Cat' and label.count(0) < 12498:
      label.append(0)
      input_path.append(os.path.join(f'{image_path}', class_name, path))
    elif class_name == 'Dog' and label.count(1) < 12498:
      label.append(1)
      input_path.append(os.path.join(f'{image_path}', class_name, path))
    elif class_name == 'Horse' and label.count(2) < 12498:
      label.append(2)
      input_path.append(os.path.join(f'{image_path}', class_name, path))
      
print(input_path[0], label[0])

In [ ]:
print(label.count(0))
print(label.count(1))
print(label.count(2))

In [ ]:
print(input_path[-1], label[-1])
print()
print(len(input_path))

### Delete Invalid Images

In [7]:
for image in input_path:
  try:
    PIL.Image.open(image)
  except:
    print(image)
    os.remove(image)

### Create Dataframe

In [ ]:
df = pd.DataFrame()
df['images'] = input_path
df['label'] = label
df = df.sample(frac=1).reset_index(drop = True)
df.head()

## Exploratory Data Analysis

In [ ]:
from tensorflow.keras.preprocessing.image import load_img

# to display grid of images 'Dogs'
plt.figure(figsize=(25,25))
temp = df[df['label']==1]['images']
start = random.randint(0, len(temp))
files = temp[start:start+25]

for index, file in enumerate(files):
  plt.subplot(5,5, index+1)
  img = load_img(file)
  img = np.array(img)
  plt.imshow(img)
  plt.title('Dogs')
  plt.axis('off')

In [ ]:
from tensorflow.keras.preprocessing.image import load_img

# to display grid of images 'Cats'
plt.figure(figsize=(25,25))
temp = df[df['label']==0]['images']
start = random.randint(0, len(temp))
files = temp[start:start+25]

for index, file in enumerate(files):
  plt.subplot(5,5, index+1)
  img = load_img(file)
  img = np.array(img)
  plt.imshow(img)
  plt.title('Cats')
  plt.axis('off')

In [ ]:
from tensorflow.keras.preprocessing.image import load_img

# to display grid of images 'Horses'
plt.figure(figsize=(25,25))
temp = df[df['label']==2]['images']
start = random.randint(0, len(temp))
files = temp[start:start+25]

for index, file in enumerate(files):
  plt.subplot(5,5, index+1)
  img = load_img(file)
  img = np.array(img)
  plt.imshow(img)
  plt.title('Horses')
  plt.axis('off')

In [ ]:
sns.countplot(df['label'])

## Create DataGenerator for the Images

In [ ]:
df['label'] = df['label'].astype('str')
df.head()

In [14]:
# input split
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(
    rescale = 1./255, # normalization of images
    rotation_range = 40, ## augmention of images to avoid overfitting
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

val_generator = ImageDataGenerator(rescale = 1./255)

train_iterator = train_generator.flow_from_dataframe(
    train,
    x_col='images',
    y_col='label',
    target_size=(128,128),
    batch_size=512,
    class_mode='categorical')

val_iterator = val_generator.flow_from_dataframe(
    test,
    x_col='images',
    y_col='label',
    target_size=(128,128),
    batch_size=512,
    class_mode='categorical')

print(len(train_iterator))
print(len(val_iterator))

## Model Creation

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

model = Sequential([
                   
    # Option 1
    Conv2D(16, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPool2D((2, 2)),
    
    Conv2D(32, (3, 3), activation='relu'),
    MaxPool2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPool2D((2, 2)),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dense(3, activation='softmax')


    # Option 2
    # Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    # MaxPool2D((2, 2)),
    
    # Conv2D(64, (3, 3), activation='relu'),
    # MaxPool2D((2, 2)),
    
    # Conv2D(128, (3, 3), activation='relu'),
    # MaxPool2D((2, 2)),
    
    # Flatten(),
    # Dense(128, activation='relu'),
    # Dense(3, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## Fit Model

In [ ]:
history = model.fit(train_iterator, epochs=10, validation_data=val_iterator)

## Save Model

In [19]:
model.save("models/my_model.keras")

In [ ]:
import datetime

current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = f"models/my_model_{current_time}.keras"
model.save(model_name)

print(f"Model saved as: {model_name}")

## Load Model

In [67]:
from keras.models import load_model

model = load_model('models/my_model.keras')

## Visualization of Results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Accuracy Graph')
plt.legend()
plt.figure()

loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Loss Graph')
plt.legend()
plt.show()

## Classificate own pictures


### Check for Existing Images

In [ ]:
directory_path = "validation_images" 

validation_full_paths = []

if os.path.exists(directory_path):
    for i_path in os.listdir(directory_path):
        full_path = os.path.join(directory_path, i_path)
        validation_full_paths.append(full_path) 
else:
    print(f"The directory '{directory_path}' does not exist.")

for path in validation_full_paths:
    print(path)


### Import Images

In [ ]:
from keras.preprocessing import image

directory_path = "validation_images/" 

testing_array = []
full_paths = []
testing_img = []
prediction_array = []

for i_path in os.listdir(directory_path):
  try:
    full_path = directory_path +  i_path
    if os.path.isfile(full_path): 
      full_paths.append(full_path)
      img = image.load_img(full_path, target_size=(128, 128))
      testing_img.append(img)
  except Exception as e:
    print("An error occurred: " + full_path, e)

print(os.listdir(directory_path))
print(full_paths)

## Prediction

In [ ]:
iteration = 0
trueCalculation = 0
falseCalculation = 0
false_calculation_path = []
main_class_names = ['Cat', 'Dog', 'Horse']


for path in full_paths:
    img = image.load_img(path, target_size=(128,128))
    img_array = image.img_to_array(img) / 255. # convert img to Numpy-Array
    img_array = np.expand_dims(img_array, axis=0) # add dimension

    prediction = model.predict(img_array) # predict
    print(prediction)
    prediction_array.append(prediction)

    prediction_array_test = np.array(prediction)
    predicted_class = np.argmax(prediction_array_test)
    print(f"Path: {path}  => Prediction: {main_class_names[predicted_class]}")
